# Clasificación de sonido ambiente 
---
# Modelado y Evaluacion con Red Neuronal Multicapa

Precargar datos del notebook 2

In [2]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

## Construir el modelo
Construimos una red neuronal meidante un perceptrón multicapa (MLP) usando *Keras* y un backend de *Tensorflow*.

Se plantea un modelo secuencial para que podamos construir el modelo capa por capa.

Se plantea una arquitectura de modelo simple, copuesta por:
- *Capa de entrada con 40 nodos*, la que nuestra funcion MFCC de extración de caracteristicas nos devuelve un conjunto de datos de 1x40

- *Capas ocultas de 256 nodos*, estas capas tendrán una capa densa con una función de activación de tipo "ReLu", (se ha demostrado que esta función de activación funciona bien en redes neuronales). También destacar que aplicaremos un valor de abandono del 50% en nuestras dos primeras capas. Esto excluirá al azar los nodos de cada ciclo de actualización, lo que a su vez da como resultado una red que es capaz de respondr mejor a la generalización y es menos probable se produzca sobreajuste los datos de entrenamiento.

- *Capa de salida de 10 nodos*, que coinciden con el número de clasificaciones posibles. La activación es para nuestra capa de salida es softmax. Softmax hace que la salida sume 1, por lo que la salida puede interpretarse como probabilidades. El modelo hará su predicción según la opción que tenga la mayor probabilidad.

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

Using TensorFlow backend.


## Compilar el modelo
Para compilar nuestro modelo, utilizaremos los siguientes tres parámetros:

* Función de pérdida: utilizaremos `categorical_crossentropy`. Esta es la opción más común para la clasificación. Una puntuación más baja indica que el modelo está funcionando mejor.

* Métricas: utilizaremos la métrica de `accuracy` que nos permitirá ver el puntaje de precisión en los datos de validación cuando entrenemos el modelo.

* Optimizador: aquí usaremos `adam`, que generalmente es un buen optimizador para muchos casos de uso.

In [5]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [6]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

## Entrenar el modelo

Se empieza probando con un número de epocas baja y se prueba hasta ver donde alcanza un valor asintotico donde por más que subamos las epocas no conseguimos que el modelo mejore significativamente.

Por otro el tamaño del bloje debe ser suficientemente bajo, ya que tener un tamaño de lote grande puede reducir la capacidad de generalización del modelo.

In [7]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, 
          validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 32 samples, validate on 8 samples
Epoch 1/100
32/32 [==============================] - 2s 62ms/step - loss: 32.6870 - accuracy: 0.1250 - val_loss: 19.4415 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 19.44147, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/100
32/32 [==============================] - 0s 421us/step - loss: 37.4246 - accuracy: 0.1250 - val_loss: 17.4310 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 19.44147 to 17.43100, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 3/100
32/32 [==============================] - 0s 397us/step - loss: 33.7399 - accuracy: 0.2500 - val_loss: 16.3363 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss improved from 17.43100 to 16.33628, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/100
32/32 [==============================] - 0s 464us/step - loss: 30.7156 - accuracy: 0.1875 - val_loss: 15.9737 - val_accuracy: 0.0000e+00

Epoch 00004: val_l


Epoch 00033: val_loss improved from 1.28189 to 1.04310, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 34/100
32/32 [==============================] - 0s 745us/step - loss: 8.1882 - accuracy: 0.5625 - val_loss: 0.7826 - val_accuracy: 0.6250

Epoch 00034: val_loss improved from 1.04310 to 0.78261, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 35/100
32/32 [==============================] - 0s 486us/step - loss: 7.6285 - accuracy: 0.3750 - val_loss: 0.6363 - val_accuracy: 0.7500

Epoch 00035: val_loss improved from 0.78261 to 0.63634, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 36/100
32/32 [==============================] - 0s 499us/step - loss: 4.1858 - accuracy: 0.5312 - val_loss: 0.4869 - val_accuracy: 0.8750

Epoch 00036: val_loss improved from 0.63634 to 0.48688, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 37/100
32/32 [==============================] - 0s 440us/step - loss: 5.8391 - accuracy: 0.4062 - val_loss

Epoch 70/100
32/32 [==============================] - 0s 559us/step - loss: 1.7342 - accuracy: 0.7500 - val_loss: 0.0106 - val_accuracy: 1.0000

Epoch 00070: val_loss improved from 0.01483 to 0.01058, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 71/100
32/32 [==============================] - 0s 485us/step - loss: 0.9565 - accuracy: 0.7812 - val_loss: 0.0076 - val_accuracy: 1.0000

Epoch 00071: val_loss improved from 0.01058 to 0.00760, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 72/100
32/32 [==============================] - 0s 512us/step - loss: 2.8394 - accuracy: 0.8125 - val_loss: 0.0061 - val_accuracy: 1.0000

Epoch 00072: val_loss improved from 0.00760 to 0.00609, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 73/100
32/32 [==============================] - 0s 458us/step - loss: 1.2159 - accuracy: 0.7500 - val_loss: 0.0051 - val_accuracy: 1.0000

Epoch 00073: val_loss improved from 0.00609 to 0.00511, saving model to saved_mod

## Evaluar el modelo
Finalmente para determiar la precisión del modelo generado llamamos a la función evaluate y le pasamos los datos de test que hemos definido previamente.

In [9]:
# Evaluating the model on the testing set
score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Testing Accuracy:  1.0
